In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import edunn as nn
from edunn import utils

In [3]:
np.set_printoptions(threshold=sys.maxsize)

# Capa Convolucional 1D

La convolución es una operación matemática fundamental en el procesamiento de señales y sistemas. Se utiliza ampliamente para el filtrado de señales, el suavizado de datos y la detección de características.

En el contexto del procesamiento de señales, la convolución 1D se utiliza para extraer ciertas características de la señal, como bordes en el caso de las imágenes, o para eliminar ruido.

$$
y[n] = (x \circledast w) = \sum_{k} w[k] \cdot x[n+k]
$$

> NOTA: a cotinuación se explica cómo realizar la capa ``Conv2d``, donde su versión en 1D se puede interpretar como un caso particular de ella, de modo que la dimensión de la altura es siempre 1 e implica que el _stride_ y _padding_ sean en ese caso 1 y 0 respectivamente.

# Capa Convolucional 2D

Siguiendo la misma idea, podemos extender el concepto de convolución sobre matrices. Es decir, una convolución en 2 dimensiones. Esto nos sirve para imágenes en escala de grises, RGB y por lotes.

## Método Forward

Una capa convolucional 2D se forma aplicando una operación de convolución 2D a la entrada, seguida de una adición de Bias si se desea. La convolución es una operación sobre dos funciones $x$ y $w$, que produce una tercera función que puede ser interpretada como una versión de la entrada $x$ "filtrada" por $w$ (también conocido como kernel). 

* Si bien la convolución se define en forma continua, a nosotros nos interesa su versión discreta.

Entonces, dado un batch de imágenes de entrada de shape $(N,C_{\text{in}},H,W)$ y un batch de filtros de shape $(M,C_{\text{out}},F,F)$, la operación de convolución se puede expresar como:

$$
y[l,m,i,j] = (x \circledast w) = \sum_{a}\sum_{b} w[m,:,a,b] \cdot x[l,:,i+a,j+b]
$$

Donde $N$ es la cantidad de imágenes del lote, $C_{\text{in/out}}$ es el número de canales de la imagen de entrada y feature map de salida (respectivamente), $M$ es la cantidad de feature maps deseados y $F$ es el tamaño del kernel.

En esta fórmula se ve que las sumas se realizan sobre las dimensiones del kernel y sobre los $C$ canales a la vez, además se asume que el **stride** (paso) es de 1 y el **padding** (relleno) es de 0. El término de bias se suma después de la operación de convolución.

* Las funciones de activación pueden funcionar de manera matricial, aplicándose a cada valor de $y[k,i,j]$ sin necesidad de cambio alguno.

|||
|:-:|:-:|
|![conv2d_forward.gif](img/conv2d_forward.gif)|![conv2d_multilayer.gif](img/conv2d_multilayer.gif)|


### Hiperparámetros

El tamaño de la matriz resultante $y$ y está estrictamente relacionado por los siguientes parámetros:

* `kernel_size`: es el tamaño del filtro utilizado.
* `stride`: es el número de saltos que da el filtro cada vez que se aplica. 
* `padding`: es la cantidad de píxeles rellenos con cero en los bordes.
   * Aplicar el filtro de forma discreta ocasiona dos problemas:
       * Pérdida de información en los bordes.
       * Reducción del tamaño final del vector.
   <!-- * Se utiliza para obtener una imagen resultante de un tamaño buscado. -->

|strides|padding|
|:-:|:-:|
|![conv2d_strides.gif](img/conv2d_strides.gif)|![conv2d_padding.gif](img/conv2d_padding.gif)|

### Shapes

El tamaño de la salida $y$ (es decir, los feature maps resultantes) depende del tamaño $H_{in} \times W_{in}$ de la imagen de entrada, el tamaño del kernel $F$, el stride (paso) $S$ y el padding (relleno) $P$. La fórmula general para calcular la altura $H_{out}$ y el ancho $W_{out}$ de la salida en una capa convolucional es:

$$
\begin{aligned}
A_{out} &= \left\lfloor \frac{A_{in} - F + 2P}{S} \right\rfloor + 1 & \text{donde $A \in \{W, H\}$} \\
\end{aligned}
$$

Por lo tanto, el shape de la salida $y$ es $(N, M, H_{out}, W_{out})$. El shape del bias depende únicamente del número de feature maps $M$. Notar que:

- El tamaño del batch en la salida es igual al tamaño del batch en la entrada. Esto se debe a que cada imagen en el batch se procesa de forma independiente.
- El tamaño de los canales en la salida es igual al tamaño del batch en el kernel. Esto se debe a que cada canal de salida se genera al convolucionar la entrada con un kernel diferente.


### Implementación

Similarmente como sucede con el Modelo Lineal, en lugar de ver a $y$ como $x \circledast w + b$, podemos verlo como `x -> Convolution -> Bias -> y`. Es decir, como una sucesión de capas, donde cada una transforma la entrada `x` hasta obtener la salida `y`.

En términos de código, el método `forward` de un modelo de `Convolution` es la composición de las funciones `forward` de las capas `Convolution` y `Bias`:

```python
y_conv = conv2d(x)  # Implementar la función dentro del mismo modelo
y = bias.forward(y_conv)
```

> TIP: la operación de convolución también se puede representar como la **cross-correlation** entre $x$ y $\text{rot}_{180^\circ } \left \{ w \right \}$, es decir:
> $$
y = \left( \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix}  \circledast \begin{bmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \end{bmatrix} \right)
\Leftrightarrow
\left( \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix} \star \begin{bmatrix} w_{22} & w_{21} \\ w_{12} & w_{11} \end{bmatrix} \right) = y
$$

Implementa el método `forward` del modelo `Convolution` en el archivo `edunn/models/convolution.py`.

In [4]:
np.random.seed(123)

x = np.random.rand(2,3,7,7)
w = np.random.rand(4,3,5,5)
                       
stride,padding=2,1
kernel_initializer = nn.initializers.Constant(w)

layer=nn.Conv2d(x.shape[1],w.shape[0],kernel_size=w.shape[2:],stride=stride,padding=padding,kernel_initializer=kernel_initializer)

In [5]:
y = layer.forward(x)
y, y.shape

(array([[[[13.67533187, 15.06363019, 10.86259051],
          [18.23903647, 18.76882808, 13.67438621],
          [13.09918212, 15.0233942 , 10.85365125]],
 
         [[12.01869291, 14.31386561, 10.51646907],
          [16.51396448, 18.93949341, 13.34667392],
          [13.17224211, 17.20560982, 11.59793342]],
 
         [[12.78086503, 14.91756095, 11.60440143],
          [17.35654321, 19.83030544, 15.04708379],
          [13.83003073, 15.60867416, 12.57713001]],
 
         [[12.73609136, 15.33298352, 10.88211767],
          [15.71015046, 20.29708406, 13.69851101],
          [13.76077383, 17.26098446, 11.3925364 ]]],
 
 
        [[[13.5167941 , 15.23910784, 12.46319048],
          [14.68162586, 18.6930386 , 13.99787793],
          [11.41059579, 13.86668801, 11.81574336]],
 
         [[12.04153096, 15.6246846 , 10.85810122],
          [14.61019629, 17.68460619, 15.09498775],
          [11.48820518, 13.60652163, 11.5175481 ]],
 
         [[11.95764005, 14.67584135, 11.51677327],
          

## Método Backward

Conociendo el gradiente de la función de error con respecto a los píxeles de salida, debemos calcular el gradiente total del error con respecto a los píxeles de entrada y los pesos de los filtros. Utilizando la regla de la cadena y derivadas parciales podemos llegar a las expresiones:

$$
\frac{\partial E}{\partial x_{mn}}=\sum_{ij}\frac{\partial E}{\partial y_{ij}}\frac{\partial y_{ij}}{\partial x_{mn}} 
\quad\quad\quad
\frac{\partial E}{\partial w_{mn}}=\sum_{ij}\frac{\partial E}{\partial y_{ij}}\frac{\partial y_{ij}}{\partial w_{mn}}
$$

Donde cada píxel $x_{mn}$ de la entrada y del filtro $w_{mn}$ contribuye a uno o más píxeles $y_{ij}$ de salida, siendo solo los píxeles de salida que aparecen en las ecuaciones anteriores para un dado $x$ o $w$ los que contribuyen durante el cálculo del forward.


### SIN stride o padding

La explicación de su cálculo es sencilla a partir de un ejemplo, supongamos que $x \in \mathbb{R}^{(3\times 3)}$ y $w \in \mathbb{R}^{(2\times 2)}$, de modo los $y_{ij}$ se definen como:

$$\begin{aligned}
y_{11} &= x_{11}w_{11} + x_{12}w_{12} + x_{21}w_{21} + x_{22}w_{22} \\
y_{12} &= x_{12}w_{11} + x_{13}w_{12} + x_{22}w_{21} + x_{23}w_{22} \\
y_{21} &= x_{21}w_{11} + x_{22}w_{12} + x_{31}w_{21} + x_{32}w_{22} \\
y_{22} &= x_{22}w_{11} + x_{23}w_{12} + x_{32}w_{21} + x_{33}w_{22}
\end{aligned}$$

#### `δE/δw`

El cálculo de los gradientes del error $E$ con respecto al filtro $w$ (vector de pesos aprendido) se puede hacer derivando parcialmente como se explicó en guías anteriores:

$$
\begin{aligned}
\frac{\partial E}{\partial w_{11}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{11}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{11}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{11}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{11}}
\\
\frac{\partial E}{\partial w_{12}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{12}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{12}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{12}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{12}}
\\
\frac{\partial E}{\partial w_{21}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{21}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{21}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{21}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{21}}
\\
\frac{\partial E}{\partial w_{22}} &= \frac{\partial E}{\partial y_{11}} \frac{\partial y_{11}}{\partial w_{22}} + \frac{\partial E}{\partial y_{12}} \frac{\partial y_{12}}{\partial w_{22}} + \frac{\partial E}{\partial y_{21}} \frac{\partial y_{21}}{\partial w_{22}} + \frac{\partial E}{\partial y_{22}} \frac{\partial y_{22}}{\partial w_{22}}
\end{aligned}
$$

Donde utilizando las ecuaciones anteriormente definidas al principio de esta celda, es fácil ver que:

$$
\begin{aligned}
\frac{\partial E}{\partial w_{11}} &= \frac{\partial E}{\partial y_{11}} x_{11} + \frac{\partial E}{\partial y_{12}} x_{12} + \frac{\partial E}{\partial y_{21}} x_{21} + \frac{\partial E}{\partial y_{22}} x_{22}
\\
\frac{\partial E}{\partial w_{12}} &= \frac{\partial E}{\partial y_{11}} x_{12} + \frac{\partial E}{\partial y_{12}} x_{13} + \frac{\partial E}{\partial y_{21}} x_{22} + \frac{\partial E}{\partial y_{22}} x_{23}
\\
\frac{\partial E}{\partial w_{21}} &= \frac{\partial E}{\partial y_{11}} x_{21} + \frac{\partial E}{\partial y_{12}} x_{22} + \frac{\partial E}{\partial y_{21}} x_{31} + \frac{\partial E}{\partial y_{22}} x_{32}
\\
\frac{\partial E}{\partial w_{22}} &= \frac{\partial E}{\partial y_{11}} x_{22} + \frac{\partial E}{\partial y_{12}} x_{23} + \frac{\partial E}{\partial y_{21}} x_{32} + \frac{\partial E}{\partial y_{22}} x_{33}
\end{aligned}
$$

Tiene un patrón idéntico al de una convolución entre $x$ y $\frac{\delta E}{\delta y}$:

$$
\frac{\delta E}{\delta w} = \begin{bmatrix} x_{11} & x_{12} & x_{13} \\ x_{21} & x_{22} & x_{23} \\ x_{31} & x_{32} & x_{33} \end{bmatrix}  \circledast \begin{bmatrix} \frac{\partial E}{\partial y_{11}} & \frac{\partial E}{\partial y_{12}} \\ \frac{\partial E}{\partial y_{21}} & \frac{\partial E}{\partial y_{22}} \end{bmatrix} = x \circledast \frac{\delta E}{\delta y}
$$

#### `δE/δx`

El razonamiento para el cálculo de los gradientes del $E$ con respecto a la imagen de entrada $x$ es el mismo que antes:

$$
\begin{aligned}
\frac{\partial E}{\partial x_{11}}&=\frac{\partial E}{\partial y_{11}}w_{11}+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{12}}&=\frac{\partial E}{\partial y_{11}}w_{12}+\frac{\partial E}{\partial y_{12}}w_{11}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{13}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}w_{12}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{21}}&=\frac{\partial E}{\partial y_{11}}w_{21}+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{11}+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{22}}&=\frac{\partial E}{\partial y_{11}}w_{22}+\frac{\partial E}{\partial y_{12}}w_{21}+\frac{\partial E}{\partial y_{21}}w_{12}+\frac{\partial E}{\partial y_{22}}w_{11} 
\\
\frac{\partial E}{\partial x_{23}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}w_{22}+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}w_{11} 
\\
\frac{\partial E}{\partial x_{31}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{21}+\frac{\partial E}{\partial y_{22}}\;\:\, 0 \; 
\\
\frac{\partial E}{\partial x_{32}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}w_{22}+\frac{\partial E}{\partial y_{22}}w_{21} 
\\
\frac{\partial E}{\partial x_{33}}&=\frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+\frac{\partial E}{\partial y_{22}}w_{22} 
\end{aligned}
$$

Tal cálculo se puede automatizar mediante el operador de **full-convolution**, en la que se aplica padding a la entrada de tal manera que la dimensión de la salida sea la misma que la de la entrada. Además, para poder representarlo de tal manera, es obligatorio que la matriz de filtros $w$ se rote 180°. 

$$
\frac{\delta E}{\delta x} = \begin{bmatrix} \frac{\partial E}{\partial y_{11}} & \frac{\partial E}{\partial y_{12}} \\ \frac{\partial E}{\partial y_{21}} & \frac{\partial E}{\partial y_{22}} \end{bmatrix} \circledast \begin{bmatrix} w_{22} & w_{21} \\ w_{12} & w_{11} \end{bmatrix} = \left. \left[ \frac{\delta E}{\delta y} \circledast \text{rot}_{180^\circ } \left \{ w \right \} \right] \right|_{\text{full-padding=F-1}}
$$

![conv2d_backward.gif](img/conv2d_backward.gif)

La fórmula para lograr el padding adecuado es fácil deducirla y queda como ejercicio para el lector demostrarla (puede encontrar ayuda en el blog [cs231n](https://cs231n.github.io/convolutional-networks/)).


### CON stride y padding

La explicación de su cálculo es sencilla a partir de un ejemplo, supongamos que $x \in \mathbb{R}^{(5\times 5)}$ y $w \in \mathbb{R}^{(3\times 3)}$, con $S=2$ de modo los $y_{ij}$ se definen como:

$$\begin{aligned}
y_{11} &=x_{11}w_{11}+x_{12}w_{12}+x_{13}w_{13}+x_{21}w_{21}+x_{22}w_{22}+x_{23}w_{23}+x_{31}w_{31}+x_{32}w_{32}+x_{33}w_{33} \\
y_{12} &=x_{13}w_{11}+x_{14}w_{12}+x_{15}w_{13}+x_{23}w_{21}+x_{24}w_{22}+x_{25}w_{23}+x_{33}w_{31}+x_{34}w_{32}+x_{35}w_{33} \\
y_{21} &=x_{31}w_{11}+x_{32}w_{12}+x_{33}w_{13}+x_{41}w_{21}+x_{42}w_{22}+x_{43}w_{23}+x_{51}w_{31}+x_{52}w_{32}+x_{53}w_{33} \\
y_{22} &=x_{33}w_{11}+x_{34}w_{12}+x_{35}w_{13}+x_{43}w_{21}+x_{44}w_{22}+x_{45}w_{23}+x_{53}w_{31}+x_{54}w_{32}+x_{55}w_{33} 
\end{aligned}$$

#### `δE/δx`

Debido al tamaño de la entrada, la cantidad de derivadas parciales de $E$ respecto de los $x_{mn}$ es sustancialmente mayor a la de ejemplos previos, por lo tanto se deja oculta su demostración quedando en el lector si quiere incurrir en el _inhóspito_ cálculo de las mismas.

<details><summary>Demostración</summary>

$$
\begin{aligned}
\frac{\partial E}{\partial x_{11}}&=
    \frac{\partial E}{\partial y_{11}}w_{11}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{12}}&=
    \frac{\partial E}{\partial y_{11}}w_{12}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{13}}&=
    \frac{\partial E}{\partial y_{11}}w_{13}+
    \frac{\partial E}{\partial y_{12}}w_{11}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{14}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{12}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{15}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{13}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{21}}&=
    \frac{\partial E}{\partial y_{11}}w_{21}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{22}}&=
    \frac{\partial E}{\partial y_{11}}w_{22}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{23}}&=
    \frac{\partial E}{\partial y_{11}}w_{23}+
    \frac{\partial E}{\partial y_{12}}w_{21}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{24}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{22}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{25}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{23}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{31}}&=
    \frac{\partial E}{\partial y_{11}}w_{31}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{11}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{32}}&=
    \frac{\partial E}{\partial y_{11}}w_{32}+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{12}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{33}}&=
    \frac{\partial E}{\partial y_{11}}w_{33}+
    \frac{\partial E}{\partial y_{12}}w_{31}+
    \frac{\partial E}{\partial y_{21}}w_{13}+
    \frac{\partial E}{\partial y_{22}}w_{11} \\
\frac{\partial E}{\partial x_{34}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{32}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{12} \\
\frac{\partial E}{\partial x_{35}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}w_{33}+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{13} \\
\frac{\partial E}{\partial x_{41}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{21}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{42}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{22}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{43}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{23}+
    \frac{\partial E}{\partial y_{22}}w_{21} \\
\frac{\partial E}{\partial x_{44}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{22} \\
\frac{\partial E}{\partial x_{45}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{23} \\
\frac{\partial E}{\partial x_{51}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{31}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{52}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{32}+
    \frac{\partial E}{\partial y_{22}}\;\:\, 0 \; \\
\frac{\partial E}{\partial x_{53}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}w_{33}+
    \frac{\partial E}{\partial y_{22}}w_{31} \\
\frac{\partial E}{\partial x_{54}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{32} \\
\frac{\partial E}{\partial x_{55}}&=
    \frac{\partial E}{\partial y_{11}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{12}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{21}}\;\:\, 0 \;+
    \frac{\partial E}{\partial y_{22}}w_{33}
\end{aligned}
$$

</details>

Tales resultados siguen un patrón común, que es fácil ver si **se dilata a $\frac{\partial E}{\partial y}$ en una cantidad igual a $S-1$**, se la rellena con ceros para obtener una full-convolution y se rota el filtro $w$ en 180°.

![conv2d_dilate.gif](img/conv2d_dilate.gif)

#### `δE/δw`

Este caso tiene gran similitud al anterior en cuanto a cómo tratarlo, y el patrón se basa también en una convolución donde se debe **dilatar a $\frac{\partial E}{\partial y}$ en una cantidad igual a $S-1$**, sin necesidad alguna de realizar padding.

$$
\begin{aligned}
\frac{\partial E}{\partial w_{11}}&=\frac{\partial E}{\partial y_{11}}x_{11}+\frac{\partial E}{\partial y_{12}}x_{13}+\frac{\partial E}{\partial y_{21}}x_{31}+\frac{\partial E}{\partial y_{22}}x_{33} \\
\frac{\partial E}{\partial w_{12}}&=\frac{\partial E}{\partial y_{11}}x_{12}+\frac{\partial E}{\partial y_{12}}x_{14}+\frac{\partial E}{\partial y_{21}}x_{32}+\frac{\partial E}{\partial y_{22}}x_{34} \\
\frac{\partial E}{\partial w_{13}}&=\frac{\partial E}{\partial y_{11}}x_{13}+\frac{\partial E}{\partial y_{12}}x_{15}+\frac{\partial E}{\partial y_{21}}x_{33}+\frac{\partial E}{\partial y_{22}}x_{35} \\
\frac{\partial E}{\partial w_{21}}&=\frac{\partial E}{\partial y_{11}}x_{21}+\frac{\partial E}{\partial y_{12}}x_{23}+\frac{\partial E}{\partial y_{21}}x_{41}+\frac{\partial E}{\partial y_{22}}x_{43} \\
\frac{\partial E}{\partial w_{22}}&=\frac{\partial E}{\partial y_{11}}x_{22}+\frac{\partial E}{\partial y_{12}}x_{24}+\frac{\partial E}{\partial y_{21}}x_{42}+\frac{\partial E}{\partial y_{22}}x_{44} \\
\frac{\partial E}{\partial w_{23}}&=\frac{\partial E}{\partial y_{11}}x_{23}+\frac{\partial E}{\partial y_{12}}x_{25}+\frac{\partial E}{\partial y_{21}}x_{43}+\frac{\partial E}{\partial y_{22}}x_{45} \\
\frac{\partial E}{\partial w_{31}}&=\frac{\partial E}{\partial y_{11}}x_{31}+\frac{\partial E}{\partial y_{12}}x_{33}+\frac{\partial E}{\partial y_{21}}x_{51}+\frac{\partial E}{\partial y_{22}}x_{53} \\
\frac{\partial E}{\partial w_{32}}&=\frac{\partial E}{\partial y_{11}}x_{32}+\frac{\partial E}{\partial y_{12}}x_{34}+\frac{\partial E}{\partial y_{21}}x_{52}+\frac{\partial E}{\partial y_{22}}x_{54} \\
\frac{\partial E}{\partial w_{33}}&=\frac{\partial E}{\partial y_{11}}x_{33}+\frac{\partial E}{\partial y_{12}}x_{35}+\frac{\partial E}{\partial y_{21}}x_{53}+\frac{\partial E}{\partial y_{22}}x_{55}
\end{aligned}
$$

> NOTA: tener en cuenta que en el cálculo del backward, siempre el stride de la full-convolution se establece a 1, independientemente del stride que se utilizó en el forward. Esto se debe a que en la propagación hacia atrás se calcula el gradiente de la función de error con respecto a cada elemento de la entrada y el kernel, por lo que es necesario considerar cada posición del filtro específico sobre cada píxel de la entrada correspondiente.


In [6]:
# Define el gradiente de la salida
g = np.random.rand(*y.shape)

# Propaga el gradiente hacia atrás a través de la convolución
layer_grad = layer.backward(g)
layer_grad

(array([[[[ 0.47096459,  2.46250718,  1.18584932,  4.3152331 ,
            1.21133264,  2.61995922,  0.57393932],
          [ 2.5099557 ,  4.74033001,  4.36829141,  5.06898507,
            3.92252628,  4.77744065,  2.37148249],
          [ 1.38509306,  4.05186895,  3.28329695,  6.20316961,
            2.92940248,  4.49417376,  1.99486303],
          [ 3.5513373 ,  7.04144284,  7.07373913,  8.84648821,
            6.22521114,  6.13075418,  3.72398508],
          [ 1.13795991,  3.92725647,  2.33774064,  6.52475077,
            2.03344522,  3.98348466,  1.7456788 ],
          [ 1.69861175,  3.17599848,  3.41770171,  6.44719723,
            3.10817693,  3.45078172,  1.64450569],
          [ 1.09218471,  1.92552303,  1.89193538,  3.87590771,
            2.13731381,  2.24024357,  1.07536877]],
 
         [[ 1.05927949,  2.73105854,  2.41090678,  3.63479426,
            2.21771671,  1.90964623,  1.6292768 ],
          [ 1.95761008,  4.06599371,  3.59801568,  6.34957478,
            3.86091117

## Comprobaciones con PyTorch

In [7]:
import torch
import torch.nn as tnn

w = torch.from_numpy(w).to(torch.double)
x = torch.from_numpy(x).to(torch.double)

conv = tnn.Conv2d(in_channels=x.shape[1], out_channels=w.shape[0], kernel_size=w.shape[-1], stride=stride, padding=padding, bias=False)
conv.weight.data = w

x.requires_grad = True
conv.weight.requires_grad = True

y_torch = conv(x)
y_torch

tensor([[[[13.6753, 15.0636, 10.8626],
          [18.2390, 18.7688, 13.6744],
          [13.0992, 15.0234, 10.8537]],

         [[12.0187, 14.3139, 10.5165],
          [16.5140, 18.9395, 13.3467],
          [13.1722, 17.2056, 11.5979]],

         [[12.7809, 14.9176, 11.6044],
          [17.3565, 19.8303, 15.0471],
          [13.8300, 15.6087, 12.5771]],

         [[12.7361, 15.3330, 10.8821],
          [15.7102, 20.2971, 13.6985],
          [13.7608, 17.2610, 11.3925]]],


        [[[13.5168, 15.2391, 12.4632],
          [14.6816, 18.6930, 13.9979],
          [11.4106, 13.8667, 11.8157]],

         [[12.0415, 15.6247, 10.8581],
          [14.6102, 17.6846, 15.0950],
          [11.4882, 13.6065, 11.5175]],

         [[11.9576, 14.6758, 11.5168],
          [13.9812, 18.7252, 14.6168],
          [11.9665, 13.9699, 12.1230]],

         [[11.4113, 15.7040, 12.2889],
          [15.4540, 16.7085, 15.2707],
          [12.4468, 13.6949, 12.9637]]]], dtype=torch.float64,
       grad_fn=<Convolut

In [8]:
utils.check_same(y_torch.detach().numpy(),y)

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [9]:
# Define el gradiente de la salida
g = torch.from_numpy(g).to(torch.double)

# Propaga el gradiente hacia atrás a través de la convolución
y_torch.backward(g)

# Imprime el gradiente de la imagen de entrada y el kernel
print("Gradiente de la entrada (δE/δx):")
print(x.grad, x.grad.shape)
print("\nGradiente del kernel (δE/δw):")
print(conv.weight.grad, conv.weight.grad.shape)

Gradiente de la entrada (δE/δx):
tensor([[[[ 0.4710,  2.4625,  1.1858,  4.3152,  1.2113,  2.6200,  0.5739],
          [ 2.5100,  4.7403,  4.3683,  5.0690,  3.9225,  4.7774,  2.3715],
          [ 1.3851,  4.0519,  3.2833,  6.2032,  2.9294,  4.4942,  1.9949],
          [ 3.5513,  7.0414,  7.0737,  8.8465,  6.2252,  6.1308,  3.7240],
          [ 1.1380,  3.9273,  2.3377,  6.5248,  2.0334,  3.9835,  1.7457],
          [ 1.6986,  3.1760,  3.4177,  6.4472,  3.1082,  3.4508,  1.6445],
          [ 1.0922,  1.9255,  1.8919,  3.8759,  2.1373,  2.2402,  1.0754]],

         [[ 1.0593,  2.7311,  2.4109,  3.6348,  2.2177,  1.9096,  1.6293],
          [ 1.9576,  4.0660,  3.5980,  6.3496,  3.8609,  4.1749,  1.7693],
          [ 1.7835,  2.5013,  4.4435,  5.8828,  3.3767,  3.4735,  2.5610],
          [ 3.6120,  5.5980,  5.5338,  9.0025,  5.3607,  5.1986,  3.0602],
          [ 1.5625,  2.6331,  3.7683,  4.7344,  3.3649,  2.8586,  2.2816],
          [ 2.4391,  3.9808,  3.3597,  6.5292,  3.4932,  4.3128, 

In [10]:
utils.check_same(x.grad.numpy(),layer_grad[0])

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [11]:
utils.check_same(conv.weight.grad.numpy(),layer_grad[1]['w'])

SUCCESS :) Arrays are equal (tolerance 1e-12)


In [12]:
samples = 100
batch_size=2
din=3 # dimensión de entrada
dout=5 # dimensión de salida
input_shape=(batch_size,din,5,5)

# Verificar las derivadas de un modelo de Convolución
# con valores aleatorios de `w`, `b`, y `x`, la entrada
layer=nn.Conv2d(input_shape[1],dout,kernel_size=(3,3))


utils.check_gradient.common_layer(layer,input_shape,samples=samples,tolerance=1e-5)    

Conv2d_1 layer:
SUCCESS 28500 partial derivatives checked (100 random input samples)


## Comprobaciones para 1D

### PyTorch

In [13]:
import torch
import torch.nn as tnn

In [14]:
stride,padding=1,0

In [15]:
x1d = torch.randn(2, 3, 7, requires_grad=True)              # (batch_size, channels,    length)

x2d = torch.from_numpy(x1d.unsqueeze(2).detach().numpy())   # (batch_size, channels, 1, length)
x2d.requires_grad=True

In [16]:
conv1d = tnn.Conv1d(x1d.shape[1], 4,     3 , stride=stride, padding=padding, bias=False)
conv2d = tnn.Conv2d(x1d.shape[1], 4, (1, 3), stride=stride, padding=padding, bias=False)
conv2d.weight.data = conv1d.weight.data.unsqueeze(2)

In [17]:
output1d = conv1d(x1d)
output2d = conv2d(x2d)

In [18]:
# (batch_size, num_filters, 1, new_length) -> (batch_size, num_filters, new_length)
output2d = output2d.squeeze(2)

In [19]:
torch.allclose(output1d, output2d, atol=1e-6)

True

In [20]:
grad_output = torch.randn_like(output1d)
grad_output_np = grad_output.unsqueeze(2).clone().numpy()

output1d.backward(grad_output.clone(), retain_graph=True)
output2d.backward(grad_output, retain_graph=True)

In [21]:
torch.allclose(x1d.grad, x2d.grad.squeeze(2), atol=1e-6)

True

In [22]:
torch.allclose(conv1d.weight.grad, conv2d.weight.grad.squeeze(2), atol=1e-6)

True

### edunn

In [23]:
x = x2d.detach().numpy()
w = conv2d.weight.data.detach().numpy()
                       
kernel_initializer = nn.initializers.Constant(w)

layer=nn.Conv2d(x.shape[1],w.shape[0],kernel_size=w.shape[2:],stride=stride,padding=padding,kernel_initializer=kernel_initializer)

In [24]:
y = layer.forward(x)
y, y.shape

(array([[[[-0.12993404,  0.11155666, -0.0033199 , -0.83892553,
           -0.67269841]],
 
         [[-0.00760235,  0.24050955, -0.00378796,  0.37707243,
           -0.04805325]],
 
         [[-0.19057056, -0.57876464, -0.04827882,  0.31562192,
            0.48772967]],
 
         [[-0.26412817, -0.23179797,  0.27678517, -0.55800464,
            0.63027741]]],
 
 
        [[[-0.28622939, -0.49418893,  0.52484418,  0.10342817,
           -0.62401949]],
 
         [[-0.12532506,  0.8769521 , -0.27777117,  0.24268897,
            0.66836041]],
 
         [[-0.37239041,  0.59117104,  0.84326629, -0.96926016,
            0.00422337]],
 
         [[-0.36526372, -0.06361786,  0.56702302, -0.87976077,
            0.47470151]]]]),
 (2, 4, 1, 5))

In [25]:
utils.check_same(output2d.detach().numpy(),y.squeeze(2), tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [26]:
g = grad_output_np

layer_grad = layer.backward(g)
layer_grad

(array([[[[ 0.06686295, -0.25877774, -0.13272792,  0.30136907,
           -0.9358317 ,  0.05339767,  0.43975016]],
 
         [[-0.16607478, -0.22734913, -0.5673997 , -0.22770661,
           -0.29150054, -0.42387944, -0.07001044]],
 
         [[ 0.08155264,  1.0059274 , -0.5196476 ,  1.0666201 ,
            0.01007864, -0.8044203 , -0.28168172]]],
 
 
        [[[-0.10991132, -0.3279412 , -0.08680366,  0.24631149,
           -0.74884623, -0.10592665,  0.49745342]],
 
         [[ 0.09700203,  0.2232698 , -0.45012352, -0.08179605,
            0.29186544,  0.01312634,  0.13386849]],
 
         [[ 0.00215698, -0.62478334, -0.26635212,  0.44134447,
            0.04348988, -0.7194589 , -0.5668802 ]]]], dtype=float32),
 {'w': array([[[[ 1.03151855, -2.24804456,  1.51494894]],
  
          [[ 0.79247528, -0.07881346, -2.83958238]],
  
          [[ 0.4871261 , -1.17990772, -0.02736071]]],
  
  
         [[[ 3.80931274, -2.10681816, -1.1472133 ]],
  
          [[ 1.78010317,  0.12645573, -5.11481

In [27]:
utils.check_same(x2d.grad.squeeze(2).detach().numpy(),layer_grad[0].squeeze(2), tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)


In [28]:
utils.check_same(conv2d.weight.grad.squeeze(2).detach().numpy(),layer_grad[1]['w'].squeeze(2), tol=1e-5)

SUCCESS :) Arrays are equal (tolerance 1e-05)
